<a href="https://colab.research.google.com/github/YWchris/Simulation-based-Inference/blob/main/OOD_SBI%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
mist_eeptrk = '/content/drive/MyDrive/MIST_1.2_EEPtrk.h5'
nnfile = '/content/drive/MyDrive/nn_c3k.h5'

Mounted at /content/drive


In [ ]:
!pip install sbi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.8/732.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nflows: filename=nflows-0.14-py3-none-any.whl size=53654 sha256=5e9ed656b7a4308ed548bcfaac563cccb1e2f981b784811d943f595b0ca0475e
  Stored in directory: /root/.cache/pip/wheels/ca/8f/ac/c324eb57b461632081812c33b13161878290d0e6fbb8f5a7e2
Successfully built nflows


In [ ]:
import torch

from sbi.inference import SNPE, SNLE, prepare_for_sbi, simulate_for_sbi
from sbi.utils.get_nn_models import posterior_nn
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.analysis import pairplot, conditional_pairplot, conditional_corrcoeff
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation, rc
from IPython.display import HTML, Image

In [ ]:
from __future__ import print_function, division
from six.moves import range
import sys
import numpy as np
import scipy
import matplotlib
from matplotlib import pyplot as plt
import h5py

In [ ]:
!pip install astro-brutus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 31.3 MB/s eta 0:00:00


In [ ]:
import brutus

In [ ]:
from brutus import seds
from brutus import filters

# all filters
print('All bands:', filters.FILTERS,'\n')

# just Pan-STARRS and 2MASS
filt = filters.ps[:-2] + filters.tmass
print('Current subset:', filt)

All bands: ['Gaia_G_MAW', 'Gaia_BP_MAWf', 'Gaia_RP_MAW', 'SDSS_u', 'SDSS_g', 'SDSS_r', 'SDSS_i', 'SDSS_z', 'PS_g', 'PS_r', 'PS_i', 'PS_z', 'PS_y', 'PS_w', 'PS_open', 'DECam_u', 'DECam_g', 'DECam_r', 'DECam_i', 'DECam_z', 'DECam_Y', 'Bessell_U', 'Bessell_B', 'Bessell_V', 'Bessell_R', 'Bessell_I', '2MASS_J', '2MASS_H', '2MASS_Ks', 'VISTA_Z', 'VISTA_Y', 'VISTA_J', 'VISTA_H', 'VISTA_Ks', 'UKIDSS_Z', 'UKIDSS_Y', 'UKIDSS_J', 'UKIDSS_H', 'UKIDSS_K', 'WISE_W1', 'WISE_W2', 'WISE_W3', 'WISE_W4', 'Tycho_B', 'Tycho_V', 'Hipparcos_Hp', 'Kepler_D51', 'Kepler_Kp', 'TESS'] 

Current subset: ['PS_g', 'PS_r', 'PS_i', 'PS_z', 'PS_y', '2MASS_J', '2MASS_H', '2MASS_Ks']


In [ ]:
sedmaker = seds.SEDmaker(filters=filt, nnfile=nnfile, mistfile=mist_eeptrk)

Filters: ['PS_g', 'PS_r', 'PS_i', 'PS_z', 'PS_y', '2MASS_J', '2MASS_H', '2MASS_Ks']
Constructing MIST library...done!
Computing age weights for track (mini, feh, afe) = (300.0, 0.5, 0.0)          
Initializing FastNN predictor...done!


In [ ]:
# Define the prior
num_dim = 6
prior = utils.BoxUniform(low=torch.tensor([0.5, -2, 2.02, 0, 2.3, 0]), high=torch.tensor([2, 0.5, 8.08, 2, 4.3, 2]))

### generate data with large error (SBI++)

In [ ]:
# Given sigma_ori
sigma_ori = 0.02

# Function to generate random errors from a uniform distribution [0.02, 0.1]
def generate_random_errors(length):
    return np.random.uniform(0.02, 0.1, length)

# Revised simulator function with specified sigma_obs for each output
def simulator_big_noise(params):
    mini, feh, eep_100, av, rv, log_dist = params
    # Perform simulation or computation to obtain mags
    mags, params_prim, params_sec = sedmaker.get_sed(mini=mini, feh=feh,
                                                     eep=eep_100 * 100, smf=0.,
                                                     av=av, rv=rv,
                                                     dist=10)

    # Generate random errors with the same length as mags
    random_errors = generate_random_errors(len(mags))

    # Calculate sigma_extra using the formula
    sigma_extra = np.sqrt(random_errors**2 - sigma_ori**2)

    # Return the simulated magnitudes with the random errors and distance correction
    return mags + np.full_like(mags, 5 * (log_dist - 1)) + np.random.randn(len(mags))* random_errors, sigma_extra


### generate data with 0.02 error (x_true)

In [ ]:
def simulator_noise02(params):
    mini, feh, eep_100, av, rv, log_dist = params
    # Perform simulation or computation to obtain mags
    mags, params_prim, params_sec = sedmaker.get_sed(mini= mini, feh=feh,
                                                     eep=eep_100 * 100, smf=0.,
                                                     av=av, rv=rv,
                                                     dist=10)

    # Return the simulated magnitudes
    return mags + np.full_like(mags, 5*(log_dist -1)) + np.random.randn(len(mags)) * 0.02

### generate data with no noise (used for posterior predictive checks)

In [ ]:
def simulator_no_noise(params):
    mini, feh, eep_100, av, rv, log_dist = params
    # Perform simulation or computation to obtain mags
    mags, params_prim, params_sec = sedmaker.get_sed(mini=mini, feh=feh,
                                                     eep=eep_100 * 100, smf=0.,
                                                     av=av, rv=rv,
                                                     dist=10)



    # Return the simulated magnitudes with the random errors and distance correction
    return mags + np.full_like(mags, 5 * (log_dist - 1))

In [ ]:
simulator_big_noise, prior = prepare_for_sbi(simulator_big_noise, prior)
simulator_noise02, prior = prepare_for_sbi(simulator_noise02, prior)
simulator_no_noise, prior = prepare_for_sbi(simulator_no_noise, prior)

/usr/local/lib/python3.10/dist-packages/sbi/utils/user_input_checks.py:515: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.as_tensor(simulator(theta), dtype=float32)


In [ ]:
np.random.seed(123)
theta_int, x_int = simulate_for_sbi(simulator_noise02, prior, num_simulations=100000) # generate x_intrinsic
k = torch.any(torch.isnan(x_int), axis = 1)
x_int = x_int[~k] # delete the invalid data
theta_int = theta_int[~k] # delete the corrsponding invalid theta

Running 100000 simulations.:   0%|          | 0/100000 [00:00<?, ?it/s]

In [ ]:
np.random.seed(123)
theta, x_both = simulate_for_sbi(simulator_big_noise, prior, num_simulations=100000) # generate x_noisy
separated_data = torch.split(x_both, split_size_or_sections=1, dim=1) # split x_obs and sigma_extra (extra noise than 0.02)
sigma_extra = separated_data[1] # sigma extra (addition to 0.02)
x_noi = separated_data[0].squeeze(1) # noisy SED (with large error)
x_noi.shape

Running 100000 simulations.:   0%|          | 0/100000 [00:00<?, ?it/s]

torch.Size([100000, 8])

In [ ]:
parameter = torch.tensor(np.random.normal(loc=0, scale=1, size=(len(x_int), 6)), dtype=torch.float32) # Generate random parameters from a normal distribution
parameter.shape # Z ~ N(0,1)

torch.Size([80102, 6])

In [ ]:
# initiate inference object
from sbi.utils.get_nn_models import (
    posterior_nn,
)

density_estimator_build_fun = posterior_nn(
    model="nsf", hidden_features=60, num_transforms=5
)
inference = SNPE(prior=prior, density_estimator=density_estimator_build_fun)
inference.append_simulations(x_int, parameter) # here we want to get p(x_int|z)

In [ ]:
density_estimator = inference.train()

 Training neural network. Epochs trained: 8

In [ ]:
posterior = inference.build_posterior(density_estimator)

In [ ]:
save_posterior = '/content/drive/MyDrive/posterior_Aug19'

import pickle

with open(save_posterior, 'wb') as f:
  pickle.dump(posterior, f) # save posterior

In [ ]:
samples = posterior.sample((10000,), parameter[2]) # generate samples from p(x_int|z)

In [ ]:
log_probability = posterior.log_prob(samples, parameter[2]) # get the log_likelihood of x_int
print(log_probability)